# Parameter Passing

Bridgic can orchestrate workflows composed of workers. There are three ways to pass data among workers, including Args-Mapping, Args-Injection, Inputs-Propagation. Now let's understand them with a specific example.

Query expansion is a common step in RAG and can enhance the quality of RAG. To enhance the quality of query expansion, developers often first extract the entity information from the question and use it to assist the model in expanding the original question. Now let's take it as an example. The user inputs the original question, and then we expand the question to obtain more sub-questions. There are three steps to complete query extension:

1. Receive the user's input and perform preprocessing, get original question.
2. Extract the entity information from the question, get entity information.
3. Expand and obtain multiple queries.

Before we start, let's import the necessary packages and set the key for model invocation.

In [ ]:
import os
from bridgic.core.automa import GraphAutoma, worker, ArgsMappingRule
from bridgic.llms.openai_like.openai_like_llm import OpenAILikeLlm, Message, Role


_api_base = os.environ.get("OPENAI_LIKE_API_BASE")
_api_key = os.environ.get("OPENAI_LIKE_API_KEY")
_model_name = os.environ.get("OPENAI_LIKE_MODEL_NAME")

Now, let's implement this query expansion. We assume that the user query we receive is in this format:

```json
{
    "query": "Q: Are there any new movies that are worth watching recently?",
    "date": "2025-09-30"
}
```

In [ ]:
class QueryExpansion(GraphAutoma):
    @worker(is_start=True)
    async def start(self, query_obj: dict):
        """
        Receive the user's input and perform preprocessing, get original question.
        """
        query = query_obj["query"].split(":")[1].strip()  # preprocess the query
        return query

    @worker(dependencies=["start"], args_mapping_rule=ArgsMappingRule.AS_IS)
    async def extract_entity(self, question: str):
        """
        Extract the entity information from the question, get entity information.
        """
        llm = OpenAILikeLlm(
            api_base=_api_base,
            api_key=_api_key,
        )
        response = llm.chat(
            model=_model_name,
            messages=[Message.from_text(text="Hello, how are you?", role=Role.USER)],
        )

    @worker(dependencies=["extract_entity"], args_mapping_rule=ArgsMappingRule.AS_IS)
    async def expand_query(self, question: str, entity: str):
        """
        Expand and obtain multiple queries.
        """
        pass
        


## 1. Args-Mapping

Args-Mapping is mainly used for parameter passing between workers that have direct process dependency relationships. For detailed information about it, please refer to the documentation.

### AS_IS mode

# 